# Implement the Game of Life

https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life

https://www.youtube.com/watch?v=C2vgICfQawE


- Initialize an empty vector of shape [100, 50, 50], where 100 is the number of steps of game of life 
dynamics and 50 x 50 is the size of the grid.

- Fill the fist frame with random 0 or 1:
    * any pixel with value 0 is dead
    * any pixel with value 1 is alive  

- The evolution rules are simple: 
    - If a pixel is alive and has 2 or 3 live neighbours survives in the next step
    - If a pixel is dead with exaclty 3 live neighbours will become alive in the next step
    - Any remaining alive pixel will die
- Repeat for all pixels (loops are ok :) )
- Repeat for all time step and watch the evolution using the `create_animation` function.

Part 2: 
- Speed you your code using either numba or by vectorinzing it with numpy 
- Measure the speed up you were able to achieve 

In [ ]:
import plotly.express as px
import numpy as np

def create_animation(frames, figure_size=(800, 800), frame_rate=30):
    assert frames.ndim == 3, f"frames shape is {frames.shape} but should be a 3 dimension tensor"
    fig = px.imshow(frames, animation_frame=0, color_continuous_scale='gray')
    fig.update_coloraxes(showscale=False)
    fig.update_xaxes(dict(visible=False))
    fig.update_yaxes(dict(visible=False))
    fig.update_layout(width=figure_size[0], height=figure_size[1]) # size
    fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = frame_rate # frame rate
    fig.show()
    
sample_frames = np.random.randint(2, size=(100, 50, 50))
create_animation(frames = sample_frames, frame_rate=30, figure_size=(800, 800))

In [ ]:
import numpy as np
from numba import njit

def update_grid(grid_in):
    grid_out = np.zeros_like(grid_in)
    nb_struct = np.array([[-1, -1], [-1,  0],
                          [-1,  1], [0, -1],
                          [0,  1],  [1, -1],
                          [1,  0],  [1,  1]])
    shapex, shapey = grid_out.shape
    offset = 1
    
    for i in range(offset, shapex - offset):
        for j in range(offset, shapey - offset):
            # accumulate over the nb structure
            n_sum, current_state = 0, grid_in[i, j]
            for ii, jj in nb_struct:
                n_sum += grid_in[i + ii, j + jj]
            
            # compact game of life rules
            if n_sum == 3:
                grid_out[i, j] = 1
                
            elif current_state == 1 and n_sum == 2:
                grid_out[i, j] = 1
            # since we start from zeros need to kill cells
    return grid_out

In [ ]:
def run_game_of_life(initial_grid, num_iterations):
    frames = np.zeros((num_iterations,
                       initial_grid.shape[0],
                       initial_grid.shape[1]))
    frames[0] = initial_grid

    for i in range(1, num_iterations):
        frames[i] = update_grid(frames[i - 1])
    return frames

In [ ]:
%%time
initial_grid = np.random.randint(2, size=(50, 50))
frames = run_game_of_life(initial_grid=initial_grid, num_iterations=100)

In [ ]:
create_animation(frames)

In [ ]:
from scipy.ndimage import convolve

def update_grid_numpy(grid_in):
    new_grid = np.zeros_like(grid_in)
    kernel = np.ones((3, 3))
    kernel[1, 1] = 0
    n_counts = convolve(grid_in, kernel)

    for status, test in [(1, 2), (1, 3), (0, 3)]:
        status_cells = grid_in == status
        counts_test = n_counts == test
        become_alive_next_step = np.logical_and(status_cells, counts_test) # if both are true
        new_grid[become_alive_next_step] = 1
    
    return new_grid

def run_game_of_life_numpy(initial_grid, num_iterations):
    frames = np.zeros((num_iterations,
                       initial_grid.shape[0],
                       initial_grid.shape[1]))
    frames[0] = initial_grid

    for i in range(1, num_iterations):
        frames[i] = update_grid_numpy(frames[i - 1])
    return frames

In [ ]:
%%time
initial_grid = np.random.randint(2, size=(50, 50))
frames = run_game_of_life_numpy(initial_grid=initial_grid, num_iterations=100)

In [ ]:
create_animation(frames)

In [ ]:
from utils import read_rle, add_pattern
pattern = read_rle('./data/piship1.rle')

initial_grid = np.zeros((150, 150))
initial_grid = add_pattern(initial_grid, pattern, pos=[150//2, 150//2], centered=True)
frames = run_game_of_life(initial_grid=initial_grid, num_iterations=100)

In [ ]:
create_animation(frames)

# Tracking Highway video

- Load the `highway.npy` file. It contains 500 frames of a video surveliance camera
- Find the background by averaging all frames
- Find the absolute difference between the frames and the background
- threshold the result to obtain a binary mask of the cars
- Use morphological operations to fill the holes in the masks
- Use the label image to assign an id to each detected car

In [ ]:
from skimage.morphology import erosion, dilation, closing, opening, disk, label

In [ ]:
video = np.load('./data/highway.npy')

In [ ]:
px.imshow(video[0], color_continuous_scale='gray')

In [ ]:
backgrond = np.mean(video, axis=0)

In [ ]:
px.imshow(backgrond, color_continuous_scale='gray')

In [ ]:
detections_raw = np.abs(video - backgrond)
detections_mask = detections_raw > 0.3

In [ ]:
fig = px.imshow(detections_raw[0], color_continuous_scale='gray')
fig.show()
fig = px.imshow(detections_mask[0], color_continuous_scale='gray')
fig.update_layout(width=500, height=500) # size

In [ ]:
detections_mask_refined = closing(detections_mask[0], disk(5))
detections_mask_refined = opening(detections_mask_refined, disk(3))
detections_labels = label(detections_mask_refined)

In [ ]:
px.imshow(detections_labels)

In [ ]:
detections_labels = []
for mask in detections_mask:
    detections_mask_refined = closing(mask, disk(5))
    detections_mask_refined = opening(detections_mask_refined, disk(3))
    d_label = label(detections_mask_refined)
    detections_labels.append(d_label)
detections_labels = np.array(detections_labels)

In [ ]:
create_animation(detections_labels)